<div style="font-family: 'Times New Roman'; font-size: 16px;">

# Quantum Linear Algebra
The Harrow–Hassidim–Lloyd (HHL) algorithm [Project 1]  
*By ____, ____, ____*

</div>


<div style="font-family: 'Times New Roman'; font-size: 16px;">

This notebook will discuss the Harrow-Hassidim-Lloyd (HHL) quantum algorithm, meant for solving a linear system
$$A\vec{x}=\vec{b} \text{ ,  where } A \text{ is a hermitian matrix}$$
and where $\vec{x}$ and $\vec{b}$ ultimately represent quantum states $\ket{x}$ and $\ket{b}$ respectively. We will provide a derivation, implementation, generalization to non-hermitian matrices, as well as the context for HHL as a subroutine.

</div>

<div style="font-family: 'Times New Roman'; font-size: 16px;">

### Section 1: Mathematical Derivation
 
Our problem is represented as 
$$A\ket{x} = \ket{b}$$
where $\ket{b} \in \mathbb{C}^N$ is some given quantum state and $A \in \mathbb{C}^{N\times N}$. Our goal is to solve for $\ket{x} \in \mathbb{C}^N$ under a few conditions:
- $A$ is hermitian such that $A = A^\dagger$
- $A$ is $s$-sparse and well conditioned, meaning it has at most $s$ nonzero entries per row and its condition number  $\kappa(A)$ is relatively small such that the system is stable and less sensitive to perturbations.
We will break down the derivation into several steps.

<br>

#### **1. Rewriting**
Given that $A$ is a hermitian matrix, there exists a spectral decomposition such that the matrix can be diagonalized by unitary transformations.
$$A = UDU^T,\quad \text{ where } U \text{ is a unitary matrix and } D \text{ is diagonal composed of the real eigenvalues of } A$$
Since the columns of $U$ form an orthonormal basis and are the eigenvalues, we can rewrite this decomposition where $|u_{i}\rangle$ is the $i^{th}$ eigenvector of $A$ with respective eigenvalue $\lambda_{i}$ as:
$$A = \sum_{i=0}^{N-1}\lambda_{i}\ket{u_{i}}\bra{u_{i}}, \quad \lambda_{i}\in\mathbb{ R }$$
Likewise, as a linear transformation, we can write our resultant vector $\ket{b}$ in the eigenbasis of $A$.
$$\ket{b} = \sum_{i=0}^{N-1}b_{i}\ket{u_{i}}, \quad b_{i}\in\mathbb{ C }$$
Thus, our problem can now be rewritten as:
$$\ket{x} = A^{-1}\ket{b} = \sum_{i=0}^{N-1}\frac{1}{\lambda_{i}}b_{i}\ket{u_{i}}\bra{u_{i}}u_{i}\rangle = \sum_{i=0}^{N-1}\frac{1}{\lambda_{i}}b_{i}\ket{u_{i}}$$

<br>

#### **2. Quantum Phase Estimation**
The problem first boils down to being able to decompose a matrix to find its eigenvalues and eigenvectors in a computationally efficient manner. At a high level, Quantum Phase Estimation is a procedure that performs a series of controlled-$U$ gates given some unitary matrix $U$ with eigenvalues of the form $e^{2\pi i \theta}$, and finds $\theta$.

For HHL, first, we start with two registers $\ket{0}^{\otimes N}$ and $\ket{b}$. Our initial state is thus $\ket{\Phi_0} = \ket{0}^{\otimes N} \otimes \ket{b}.$ Then we can characterize the behavior of **QPE** as:
$$\textbf{QPE}(U,\ket{0}^{\otimes N},\ket{b}) = \textbf{QPE}(U,\ket{0}^{\otimes N},\sum_{j=0}^{N-1}b_{j}\ket{u_{j}}) = \sum_{j=0}^{N-1}\ket{\lambda_{j}}b_{j}\ket{u_{j}}, \quad\text{ for some clever choice of } U$$

We will choose $U = e^{iAt}$ for some constant $t$ such that $U$ is governed by the same eigenvalues of $A$ and we have that, where $|u_{i}\rangle$ is the $i^{th}$ eigenvector of $A$ with respective eigenvalue $\lambda_{i}$:
$$U = e^{iAt} = \sum_{j=0}^{N-1}e^{i\lambda_{j}t}\ket{u_j}\bra{u_j}$$

>1. First, we hadamard our first register of $\ket{0}^{\otimes N}$. $H^{\otimes N}\ket{\Phi_0} = \frac{1}{2^{n/2}}(\ket{0} + \ket{1})^{\otimes N} \otimes \ket{b}$
>2. Next, we perform a series of $N$ controlled-$U$ gates such that for $k = {0, 1,...,N-1}$ (notice $|k| = N$), we create the gate where $\ket{k}$ is the k-th qubit in the first registrar: 
$$CU^{2^k}(\ket{k}, \ket{b}) =_{\text{if} \ket{k}=\ket{1}} \ket{k} \otimes U^{2^k}\ket{b} = \ket{k} \otimes \sum_{j=0}^{N-1}e^{i\lambda_{j}t2^k}b_{j}\ket{u_{j}}\bra{u_{j}}u_{j}\rangle = \ket{k} \otimes \sum_{j=0}^{N-1}e^{i\lambda_{j}t2^k}b_{j}\ket{u_{j}} $$
The composition of all N of these such gates yields the state (with normalization) where $k = {2^0, 2^1,...,2^{N-1}}$:
$$ \sum_{j=0}^{N-1} \left( \left( \frac{1}{\sqrt{2^n}} \sum_{k=0}^{2^{n-1}} e^{i k \lambda_j t} \ket{k} \right) \otimes b_j \ket{u_j} \right)$$
 >3. Now, it is clear that the phase factor actually depends on the control qubits, and so we consider just our first (control) register, which is now superposed through unitary entanglements to the state:
 $$ \frac{1}{\sqrt{2^n}} \sum_{k=0}^{2^{n-1}} e^{i k \lambda_j t} \ket{k} $$
 $$ \text{where the phase factor is }\quad \lambda_i t = 2\pi\theta $$
 It is clear now that the encoded phase factor we desire (which represents the eigenvalues of $A$), can be extracted through the inverse quantum fourier transform operation. Thus, applying **QFT$^{-1}$** has the following behavior:
 $$\textbf{QTF}^{-1}(\frac{1}{\sqrt{2^n}} \sum_{k=0}^{2^{n-1}} e^{i k \lambda_j t} \ket{k}) \mapsto \ket{\theta} \quad\text{ which encodes } \theta_j = \lambda_j t / 2\pi$$
 Thus, for our total state, we are left with:
 $$\sum_{j=0}^{N-1}\ket{\lambda_j}\otimes b_{j}\ket{u_j}$$

<br>

#### **3. Quantum Phase Estimation**
With our new intermediate state $\sum_{j=0}^{N-1}\ket{\lambda_{j}} \otimes b_{j}\ket{u_{j}} = \ket{\lambda} \otimes \ket{b}$, our next problem is to "invert the eigenvalue." We will do this by applying a series of controlled rotations. These rotations will utilize the control register which now encodes the eigenvalues applied onto a new ancilla register that will extract $\frac{1}{\lambda}$. (Checkpoint: by now, we have created 3 registers with a total state of $\ket{\lambda} \otimes \ket{b} \otimes \ket{0}_{ancilla}$

<br>

#### **4. Uncompute**


</div>